<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2025notebooks/2025_1024word2vec_jalex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vec (2025) 実習

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br/>
Date: 24/Oct/2025<br/>
Original Date: 19/Jun/2020<br/>
MIT license
</div>



<center>

<img src="https://komazawa-deep-learning.github.io/assets/2013Mikolov_KingQueenFig.svg" width="600"><br/>
From Mikolov et. al (2013) Linguistic Regularities in Continuous SpaceWord Representations, Fig. 2<br/>
</center>
全ての概念がある多次元空間内に存在するとします。各概念はこの空間内での位置ベクトルとして表現されます。
このとき，高等学校のときのベクトルの知識を思い出すと，$\mathbf{a}-\mathbf{b}$ は，ベクトル $\mathbf{b}$ から ベクトル $\mathbf{a}$ へ向かうベクトルとして表されます。
すなわち女王ベクトルから王様ベクトルを引くということは，王様を原点と考えて，女王を眺めた方向を表すベクトルと考えることができます。
このような，眺める方向を表すベクトルと，別のベクトル 女性ベクトルから男性ベクトルを引いたベクトル，
すなわち，男性を原点として女性を眺めたベクトルを考えます。
このベクトル 2 つ，<b><font color="teal">女王 - 王様</font></b> と <b><font color="teal">女 - 男</font></b> がほぼ重なることを
比喩的に <b><font color="teal">女王 - 王様 = 女 - 男</font></b> となります。この式を少し変形すれば，
<b><font color="teal">王様 - 男 + 女 = 女王</font></b> となるでしょう。これを言葉で表せば，
<b><font color="red">王様から男を引いて，女を足すと女王になる</font></b>，という演算が形式的に成り立ちます。

<br/>
<br/>
<center>
<img src="https://komazawa-deep-learning.github.io/assets/2013Mikolov_FigCountries.svg" width="680"><br/>
Modified from Mikolv et. al (2013) Distributed Representations of Words and Phrases and their Compositionality, Fig. 2</br/>

<br/><br/>
<img src="https://komazawa-deep-learning.github.io/assets/2013Mikolov_Fig1.svg" width="840"><br/>
</center>



$$
\ell=\frac{1}{T}\sum_{t=1}^{T}\sum_{-c\ge j\ge c,j\ne0}\log p\left(w_{t+1}\vert w_{t}\right),
$$

$$
p\left(w_o\vert w_i\right)=\frac{\exp(v_{w_{o}}^\top v_{w_{w_i}})}{\sum\exp(v_{w_{o}}^\top v_{w_{w_i}})}
$$

# 準備作業

In [ ]:
# 2025 年 05 月 01 日時点でのメモ
# numpy のバージョンを強制的に 1.26.4 にダウングレードした場合，ランタイムの再起動をしないと
# ダウングレードした numpy が実行時に反映されない。
# このため一度このセルを実行した後に,上のメニューバー左から 5 つ目の「ランタイム」から「セッションを再起動する」
# を選択して再度このセルを実行する必要がある。

# Google Colab 上で実行しているかどうかを判定
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

# Google colab で実行している場合必要なライブラリをインストールして word2vec ファイルをダウンロードする
if isColab:
    !pip install pydrive2

    #Import modules
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    #Authenticate and create the PyDrive client
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    #Get the Shareable link
    # 2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz: 1Rp3HbDkbpzMg5ehq1ARwCATX8iZAxTgj
    # 2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_sgns.bin.gz: 19BKVOBNHESt1K8725UTM9J3OpqK7YlVb

    # 2021_05jawiki_hid200_win20_neg20_sgns.bin.gz: 1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s
    # 2021_05jawiki_hid200_win20_neg20_cbow.bin.gz: 1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7
    # 2021_05jawiki_hid128_win10_neg10_sgns.bin.gz: 1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M
    # 2021_05jawiki_hid128_win10_neg10_cbow.bin.gz: 1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb

    downloaded = drive.CreateFile({'id':"1Rp3HbDkbpzMg5ehq1ARwCATX8iZAxTgj"})
    downloaded.GetContentFile('2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz')
    # 上記は cbow で訓練済ファイルをダウンロードしています。
    # skip gram モデルで訓練済モデルに変更する場合には，上記情報に従って適宜書き換えてください。

    !pip uninstall numpy thinc spacy --yes
    !pip install --upgrade numpy==1.26.4 --force-reinstall
    !pip install --upgrade gensim==4.3.3

    import numpy as np
    print(f'np.__version__:{np.__version__}')

    import gensim
    print(f'gensim.__version__:{gensim.__version__}')

# 訓練済データの読み込み

In [2]:
# 上セルでダウンロードした訓練済 word2vec ファイルを読み込む
word2vec_file='2017Jul_jawiki-wakati_neologd_hid200_win20_neg20_cbow.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_file,
                                                        encoding='utf-8',
                                                        unicode_errors='replace',
                                                        binary=True)

In [ ]:
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=5)

In [ ]:
model.most_similar(positive=['王','女'], negative=['男'], topn=10)

In [ ]:
# model.most_similar(positive=['日本心理学会'], negative=['心理学'], topn=10)
# model.most_similar(positive=['心理学'], negative=['科学'], topn=10)
model.most_similar(positive=['心理学'], negative=['実験'], topn=10)

In [ ]:
# model.most_similar(positive=['言語','実験'], topn=10)
model.most_similar(positive=['言語学','心理学'], topn=10)

In [ ]:
# model.most_similar(positive=['聴覚','認知科学'], topn=5)
# model.most_similar(positive=['視覚','認知科学'], topn=5)
# model.most_similar(positive=['視覚','認知心理学'], topn=5)
model.most_similar(positive=['言語','思考'], topn=5)

In [ ]:
model.most_similar(positive=['ユニクロ','錦織圭'], negative=['テニス'], topn=10)

In [ ]:
model.most_similar(positive=['ジャイアンツ','アメリカ'], negative=['日本'], topn=10)

In [ ]:
# model.most_similar(positive=['ユニクロ'], negative=['アパレル'], topn=10)
# model.most_similar(positive=['ユニクロ','アメリカ'], negative=['日本'], topn=10)
model.most_similar(positive=['ユニクロ','ソフトバンク'], negative=['アパレル'], topn=10)

In [ ]:
# model.most_similar(positive=['1','10'], negative=['+'], topn=5)
# model.most_similar(positive=['2','1'], negative=['+'], topn=5)
model.most_similar(positive=['2','1','-'], topn=5)

In [ ]:
model.most_similar(positive=['NTT', '中国'], topn=10)

In [ ]:
model.most_similar(positive=['東京', '皇居'], topn=10)
model.most_similar(positive=['ユニクロ', '楽天'], negative=['IT企業'], topn=10)
model.most_similar(positive=['ユニクロ', '無印良品'], topn=10)

In [ ]:
model.similarity('女性', '男性')

In [ ]:
model['computer']  # raw NumPy vector of a word

In [ ]:
model['コンピュータ']  # raw NumPy vector of a word

In [ ]:
model.similarity('computer', 'コンピュータ')

In [ ]:
model.similarity('女性', '女')

In [ ]:
model.most_similar(positive=['京都','皇居'], negative=['東京'], topn=5)

# Jalex の読み込み

JALEX: Japanese version of lexical decision database<br/>
Naoto Ota and Masaya Mochizuki<br/>
Date: 2025<br/>
DOI: [10.3389/flang.2024.1506509](https://www.frontiersin.org/journals/language-sciences/articles/10.3389/flang.2024.1506509/full)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls drive/Shareddrives/2024JALEX
!pwd

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())

try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

import os

if isColab:
    HOME = '/content'
    localdir = 'drive/Shareddrives/2024JALEX'
else:
    HOME = os.environ['HOME']
    localdir = 'study/2025_2014jalex'

basedir = os.path.join(HOME, localdir)
jalex_xls_fname = 'JALEX.xlsx'
df = pd.read_excel(os.path.join(basedir, jalex_xls_fname))
RT, SD = df['平均反応時間 ミリ秒'], df['反応時間の標準偏差']

print(df.columns)

bins = 60
RT.hist(bins=bins)
plt.title(f'bins:{bins}')
plt.show()


In [ ]:
df

In [ ]:
print(len(df['目標語'].to_list()))
print(df['目標語'].to_list()[:5])
jalex_wrds = df['目標語'].to_list()

In [ ]:
# JALEX 単語の中で word2vec にエントリがある単語を抽出
jalex_vecs = []
jalex_wrds_in = []
jalex_rts = []
jalex_sds = []
for i, wrd in enumerate(jalex_wrds):
    if not model.has_index_for(wrd):
        print(f'{i:08d}: {wrd} は word2vec 中に存在しない {model.has_index_for(wrd)}')
    else:
        jalex_vecs.append(model.get_vector(wrd))
        jalex_wrds_in.append(wrd)
        jalex_rts.append(RT[i])
        jalex_sds.append(SD[i])

jalex_vecs, jalex_rts, jalex_sds = np.array(jalex_vecs), np.array(jalex_rts), np.array(jalex_sds)

# 反応時間の予測

In [16]:
# 線形回帰
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
sk_model = LinearRegression()


In [ ]:
# データを分割して，訓練データと検証データとに分ける
X_train, X_test, y_train, y_test = train_test_split(jalex_vecs, jalex_rts, test_size=0.1, random_state=42)

# モデルの当てはめ
sk_model.fit(X_train, y_train)

# 予測
y_pred = sk_model.predict(X_test)

# 評価
mse = mean_squared_error(y_test, y_pred)
print(f"平均二乗誤差: {mse}")
print(f'{np.sqrt(mse):.3f}')

In [ ]:
plt.scatter(y_pred, y_pred - y_test, color = 'blue') # 残差プロット
plt.title('残差プロット')                            # タイトル
plt.xlabel('予測値')                                 # x 軸
plt.ylabel('残差')                                   # y 軸
plt.grid()                                           # グリッド線表示
plt.show()                                           # 図の表示


In [ ]:
sk_model_sd = LinearRegression()
# データを分割して，訓練データと検証データとに分ける
X_train, X_test, y_train, y_test = train_test_split(jalex_vecs, jalex_sds, test_size=0.1, random_state=42)

# モデルの当てはめ
sk_model_sd.fit(X_train, y_train)

# 予測
y_pred = sk_model_sd.predict(X_test)

# 評価
mse = mean_squared_error(y_test, y_pred)
print(f"平均二乗誤差: {mse}")
print(f'{np.sqrt(mse):.3f}')

In [ ]:
plt.scatter(y_pred, y_pred - y_test, color = 'blue') # 残差プロット
plt.title('残差プロット')                            # タイトル
plt.xlabel('予測値')                                 # x 軸
plt.ylabel('残差')                                   # y 軸
plt.grid()                                           # グリッド線表示
plt.show()                                           # 図の表示
